In [2]:
import numpy as np

print("End")

End


In [20]:
my_list = [1, 2, 3]
print(my_list * 2)


[1, 2, 3, 1, 2, 3]


In [ ]:
import requests # Library # HTTP GET request to a website 
from bs4 import BeautifulSoup

response = requests.get('https://pokeapi.co/api/v2/pokemon/pikachu')
#print(response) # Prints 'Response [200]' which means 'yes I connected to the website'
#print(response.text)
#r = requests.get('http://pythonscraping.com/pages/page3.html') 
#html_string = r.text # Returns the response as a string 
#print(r.text)

In [ ]:
# Create BeautifulSoup object 
page = requests.get('http://pythonscraping.com/pages/page3.html') 
html = page.text # GET page content 
soup = BeautifulSoup(html, 'html.parser') # Converts into a BeautifulSoup object 
print(soup.prettify())

In [ ]:
# Access a specific element 
fish = soup.find(name='tr', attrs={'id': 'gift3', 'class': 'gift'}) 
print(fish) # Returns 'None' if it cant find it

# Find all. Each element in the list is a column on the page 
fish_row = fish.find_all('td') # Returns a list of td tags 
title = fish_row[0].text 
description = fish_row[1].text 
price = fish_row[2].text 
print(title) 
print(description) 
print(price) 

parrot = fish.find_next_sibling()  
parrot_children = fish.findChildren()

In [48]:
# Create BeautifulSoup object 
url = requests.get('https://en.wikipedia.org/wiki/Python_(programming_language)') 
html = url.text # GET page content 
soup = BeautifulSoup(html, 'html.parser') # Converts into a BeautifulSoup object 